In [ ]:
import os
import cv2
import numpy as np
from PIL import Image

vis_dir = '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/train'
input_dirs = {
    'train': '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/train',
    'validation': '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation',
    'test': '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/test'
}
output_dir = '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/processed'

for subset in input_dirs.keys():
    subset_dir = os.path.join(output_dir, subset)
    if not os.path.exists(subset_dir):
        os.makedirs(subset_dir)

def apply_contour(input_path):
    img = cv2.imread(input_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def apply_blur(input_path, blur_type='gaussian'):
    img = cv2.imread(input_path)
    if blur_type == 'gaussian':
        img = cv2.GaussianBlur(img, (5, 5), 0)
    elif blur_type == 'median':
        img = cv2.medianBlur(img, 5)
    return img

def apply_hist_eq(input_path):
    img = cv2.imread(input_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eq_img = cv2.equalizeHist(gray)
    return eq_img

def apply_edge_enhancement(input_path):
    img = cv2.imread(input_path)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]) 
    sharpened = cv2.filter2D(img, -1, kernel)
    return sharpened

def resize_and_save_image(input_path, output_path, size=(128, 128)):
    try:
        with Image.open(input_path) as img:
            if img.mode == 'P':
                img = img.convert('RGBA')
            if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
                img = img.convert('RGB')

            img_array = np.array(img) 
            img_array = apply_contour(input_path)  
            img_array = apply_blur(input_path)   
            img_array = apply_hist_eq(input_path)  
            img_array = apply_edge_enhancement(input_path)  

            img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

            img = Image.fromarray(img_array)
            img = img.resize(size, Image.LANCZOS)
            img.save(output_path, format='JPEG')
    except Exception as e:
        print(f"Error processing {input_path}: {e}")

def process_directory(input_directory, output_directory):
    for root, dirs, files in os.walk(input_directory):
        relative_path = os.path.relpath(root, input_directory)
        output_path = os.path.join(output_directory, relative_path)
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        
        for file_name in files:
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_file_path = os.path.join(root, file_name)
                output_file_path = os.path.join(output_path, file_name)
                resize_and_save_image(input_file_path, output_file_path)

for subset, dir_path in input_dirs.items():
    process_directory(dir_path, os.path.join(output_dir, subset))


In [4]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle
import matplotlib.pyplot as plt

train_dir = '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/processed/train'
test_dir = '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/processed/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    seed=777,
    target_size=(128, 128),
    batch_size=20,
    class_mode='categorical'
)
import json
with open('class_indices.json', 'w') as f:
    json.dump(train_generator.class_indices, f)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    seed=777,
    target_size=(128, 128),
    batch_size=20,
    class_mode='categorical'
)

checkpoint_callback = ModelCheckpoint(
    '/content/processed/best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

Found 3114 images belonging to 36 classes.
Found 359 images belonging to 36 classes.


In [ ]:
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.GlobalAveragePooling2D(),  

    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5), 
    layers.Dense(36, activation='softmax') 
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=test_generator,
    verbose=2,
    callbacks=[checkpoint_callback, lr_scheduler, early_stopping]
)

with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

In [5]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
import os

model = tf.keras.models.load_model('/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/best_model.keras')

image_paths = [
    '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation/corn/Image_7.jpg',
    '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation/cabbage/Image_6.jpg',
    '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation/paprika/Image_10.jpg',
    '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation/orange/Image_10.png',
    '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation/orange/cam.jpg',
    '/home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation/ginger/gung.jpg'
]

def get_true_label(img_path):
    return os.path.basename(os.path.dirname(img_path))

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128)) 
    img_array = image.img_to_array(img)  
    img_array = np.expand_dims(img_array, axis=0)  
    img_array = img_array / 255.0 
    return img_array

predictions = []
true_labels = []

for img_path in image_paths:
    img_array = load_and_preprocess_image(img_path)
    pred_prob = model.predict(img_array)
    pred_class = np.argmax(pred_prob, axis=1)[0]  
    predictions.append(pred_class)
    true_labels.append(get_true_label(img_path))  

label_names = {v: k for k, v in test_generator.class_indices.items()}

print("Results:")
for i, img_path in enumerate(image_paths):
    pred_label = label_names[predictions[i]]
    true_label = true_labels[i]
    print(f"Image: {img_path}")
    print(f"Predicted Class: {pred_label}")
    print(f"True Class: {true_label}")
    print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Results:
Image: /home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation/corn/Image_7.jpg
Predicted Class: sweetcorn
True Class: corn

Image: /home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation/cabbage/Image_6.jpg
Predicted Class: cabbage
True Class: cabbage

Image: /home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation/paprika/Image_10.jpg
Predicted Class: paprika
True Class: paprika

Image: /home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation/orange/Image_10.png
Predicted Class: orange
True Class: orange

Image: /home/leo/workspace/Personal_Project/Children_Vision/models/images_classification/validation/ora